# Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# BigQuery View creation script to combine v1.0 & v1.1 report versions.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YouTubeLabs/code-samples/blob/main/create_bigquery_views/bigquery_view_creation_example.ipynb)


This colab is intended to be used with content owner
reports that were transferred into BigQuery using the [BigQuery Data Transfer
Service for YouTube Content Owners](https://cloud.google.com/bigquery/docs/youtube-content-owner-transfer). This colab provides an example on how to create a [logical view](https://cloud.google.com/bigquery/docs/views-intro) that combines a new version of a YouTube Report with a previous version. Throughout the notebook we are using the v1.1 `(_a2)` (non music) Shorts Ads Revenue Summary Report and the old v1.0 `(_a1)` report as an example, but it can be used with any report in [this list](https://developers.google.com/youtube/reporting/revision_history#june-24,-2025).

<br>

This script looks for the earliest date that data in the v1.1 report is available and then combines with data before that date from the v1.0 report to create a logical view with both reports.

<br>

This example script uses data from the following two non-music Shorts Ads Revenue reports:
- `content_owner_shorts_ad_revenue_summary_a1`
- `content_owner_shorts_ad_revenue_summary_a2`

**You can modify this colab to create logical views for different reports, just ensure that the columns in the query reflect the columns in the reports.**

<br>

To run this colab you will need the following:

1.   An active BigQuery Data Transfer config with [YouTube Content Owner Transfers](https://cloud.google.com/bigquery/docs/youtube-content-owner-transfer) in your Google Cloud Project.
2.   Your Google Cloud Project ID. You can find it on [the home page](https://console.cloud.google.com/) of your Google Cloud console.
3.   Your dataset ID where the tables from your BigQuery Transfer Jobs are located.
4.   Name that you want your new view to have.


<font color="red" size=4><strong> Use at your own risk: It is your responsibility to check that the view creation and revenue calculations done with views created in this colab are correct and that all applicable revenue categories are accounted for.</strong></font>

<br>

Your `REPORT_V0_TABLE` and `REPORT_V1_TABLE` should include the **entire table path,** including the Project ID and the Dataset ID. For example: `my_project.my_dataset.my_report_v0_table`. The script will not re-use the `PROJECT_ID` and `DATASET_ID` parameters that you have defined above, in case your tables or views are in different locations.

In [1]:
# Your Google Cloud Project ID
PROJECT_ID = "" #@param {type:'string', placeholder:'Your Google Cloud Project ID'}

# Define your target dataset and the name for the new view.
DATASET_ID = "" #@param {type:'string', placeholder:'Your Dataset ID'}

# Table Name for your Shorts Ads Revenue Summary v1.0 (_a1) table.
# Should be a partitioned table in PROJECT_ID.DATASET_NAME.TABLE_NAME format.
REPORT_V0_TABLE = "" #@param {type:'string', placeholder:'Your Report v1.0 (or _a1) table.'}

# Table Name for your Shorts Ads Revenue Summary v1.1 (_a2) table.
# Should be a partitioned table in PROJECT_ID.DATASET_NAME.TABLE_NAME format.
REPORT_V1_TABLE = "" #@param {type:'string', placeholder:'Your Report v1.1 (or _a2) table.'}

# ID of the Content Owner for which you want to calculate revenue.
VIEW_NAME = "" #@param {type:'string', placeholder:'Your New View Name'}

min_date_v1_query = f"""
SELECT MIN(TIMESTAMP(_PARTITIONTIME)) AS min_v1_data_date
FROM `{REPORT_V1_TABLE}`
WHERE _PARTITIONTIME IS NOT NULL
"""
default_switch_date = '2050-12-31'

### Authenticate your Colab environment for Google Cloud.

An OAuth pop-up window should appear for authentication.

In [ ]:
from google.colab import auth

def authenticate_to_cloud():
  try:
    auth.authenticate_user()
    print("Successfully authenticated to Google Cloud!")
  except Exception as e:
    print(f"Error authenticating to Google Cloud: {e}")

authenticate_to_cloud()

### Import Required Libaries and Initialize the BigQuery Client

In [3]:
# Import BigQuery client library and Pandas
from google.cloud import bigquery
import pandas as pd

# Initialize the BigQuery client
client = bigquery.Client(project=PROJECT_ID)

### Find the earliest date partition in your v1.1 report.

To know which date range to take from each table to create the view, we must
find the earliest date that the v1.1 report became available to avoid double counting between the two tables. This is the newer of the two reports.

In [ ]:
def find_earliest_partition(query_earliest_date):
  """Find the earliest date that the data in a table was available."""
  print(f"Attempting to find the earliest _PARTITIONTIME in: "
        f"{REPORT_V1_TABLE}")
  try:
      # Execute the query and fetch the result
      min_v1_date = client.query(min_date_v1_query)
      min_v1_date_result = min_v1_date.result().to_dataframe()

      if (not min_v1_date_result.empty and
         pd.notna(min_v1_date_result['min_v1_data_date'].iloc[0])):
          # Convert date object to string 'YYYY-MM-DD' for SQL
          switch_date = (min_v1_date_result['min_v1_data_date']
                         .iloc[0].strftime('%Y-%m-%d'))
          print(f"Date of earliest data found in {REPORT_V1_TABLE} is"
                f" {switch_date}")
          return switch_date
      else:
          # Fallback if V1.1 table is empty or _PARTITIONTIME is null
          # A far in the future means V0 will cover all data until V1 gets data.

          print(f"No valid date found in {REPORT_V1_TABLE} table. "
                f"Using default switch date {default_switch_date} instead.")
          return default_switch_date

  except Exception as e:
      # Fallback if REPORT_V1_TABLE table doesn't exist or query fails
      print(f"Error querying {REPORT_V1_TABLE} table ({e}). Using default"
            f" switch date {default_switch_date} instead.")
      return default_switch_date

switch_date = find_earliest_partition(query_earliest_date=min_date_v1_query)
print(f"Switching from {REPORT_V0_TABLE} to \n{REPORT_V1_TABLE} data at"
      f" _PARTITIONTIME >= {switch_date}")

### Construct the query to create your desired view, combining both tables.

The view query takes all v1.0 data before the new v1.1 table was created, unioning it with all data from the new v1.1 table. This will create a combined logical view with data from both tables.

In the logical view, `total_views` from the v1.0 `(_a1)` table are renamed to `engaged_views` in this example.

As mentioned above, you can modify this colab to create logical views for different reports, just ensure that the columns in the query below also reflect the columns in the available columns in the reports.

Then, we have a small query to show 5 rows from the view for you to check if the view was successfully created.


In [6]:
view_creation_query = f"""
  CREATE OR REPLACE VIEW `{PROJECT_ID}.{DATASET_ID}.{VIEW_NAME}` AS
  WITH
    ShortsRevenueV0 AS (
      SELECT
        adjustment_type_,
        video_id,
        video_title,
        video_duration_sec,
        category,
        channel_id,
        uploader,
        content_type,
        policy,
        total_views AS engaged_views,
        net_partner_revenue_post_revshare,
        DATE(_PARTITIONTIME) AS _DATA_DATE
      FROM `{REPORT_V0_TABLE}`
      -- V0 data BEFORE the switch date
      WHERE TIMESTAMP(_PARTITIONTIME) < '{switch_date}'
    ),
    ShortsRevenueV1 AS (
      SELECT
        adjustment_type_,
        video_id,
        video_title,
        video_duration_sec,
        category,
        channel_id,
        uploader,
        content_type,
        policy,
        engaged_views,
        net_partner_revenue_post_revshare,
        DATE(_PARTITIONTIME) AS _DATA_DATE
      FROM `{REPORT_V1_TABLE}`
      -- V1 data ON OR AFTER the switch date
      WHERE TIMESTAMP(_PARTITIONTIME) >= '{switch_date}'
    )
  SELECT
    *
  FROM ShortsRevenueV0
  UNION ALL
  SELECT
    *
  FROM ShortsRevenueV1;
"""

top_view_results_query = f"""
  SELECT *
  FROM `{PROJECT_ID}.{DATASET_ID}.{VIEW_NAME}`
  LIMIT 5
"""

### Create the BigQuery View

Function that creates the BigQuery view using the `view_creation_query` above.
If this function has run successfully, then you should also see this view available in your BigQuery Studio within the dataset that you've defined above.

Depending on how much data you have in the tables that you have combined, it might take some time for the query to complete.

In [ ]:
def create_bigquery_view(view_creation_query):
  """Create a view in BigQuery using the view creation query."""
  print(f"Now attempting to create or replace view: "
      f"{PROJECT_ID}.{DATASET_ID}.{VIEW_NAME}")
  try:
      # Run the query to create the view.
      query_job = client.query(view_creation_query)

      # Wait for query to finish running.
      query_job.result()

      print(f"View '{VIEW_NAME}' successfully created/updated in dataset"
            f"'{DATASET_ID}'.")
      print("You can verify its existence and data in the BigQuery UI.")
  except Exception as e:
      return (f"Error creating view: {e}")

create_bigquery_view(view_creation_query=view_creation_query)

### [Optional] Query the View to See That It Was Created Successfully

The function below will run a query to display 5 rows of data from the view that has been created above. It will display the output below the code in a data table.

In [ ]:
def query_view_results(query_string):
  """Run a sample query to display results from a view."""
  print(f"Querying the new view to see some sample data using: {query_string}.")

  try:
      # Run the query to create the view.
      query_job = client.query(query_string)

      # Convert the query results to a Pandas DataFrame.
      view_dataframe = query_job.to_dataframe()

      print("Successfully fetched the data from your view, displaying the "
      "first 5 rows below:")

      # Display 5 rows of the view.
      return display(view_dataframe.head(5))

  except Exception as e:
      print(f"An Error occured querying data using the BigQuery client: {e}")
      return ("Please check your project ID, dataset ID, view name, "
             "and/or IAM permissions.")

query_view_results(query_string=top_view_results_query)